# TP N° 1: Análisis Exploratorio de Datos
## Heart Diseases Dataset
Este conjunto de datos es una versión avanzada del clásico conjunto de datos de enfermedades cardíacas de UCI Machine Learning, enriquecido con más características para soportar análisis más sofisticados.

## 1- Listado de variables y selección 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import h5py
import PIL
import seaborn as sns
import plotly
import sklearn_pandas

In [ ]:
data = pd.read_csv('./heart_disease_data_with_features.csv')

In [ ]:
data.shape
# Muestra las dimensiones del dataframe

In [ ]:
data.head(10)

In [ ]:
# Dividimos por las columnas de interes
subset = data.loc[:, ['sex','cp','fbs','restecg','thalach', 'exang','oldpeak','slope', 'ca','thal', 'num','age_group', 'cholesterol_level', 'bp_level','risk_score', 'symptom_severity', 'risk_factor', 'avg_chol_by_age_group']]
subset.describe()

In [ ]:
subset.head(10)

In [ ]:
# Mostramos los tipos de datos
subset.dtypes

In [ ]:
# Elije 10 filas al azar del DataFrame
subset.sample(10)

In [77]:
import plotly.express as px

fig = px.box(subset, y='thalach', )
fig.show()

Analizando la variable "thalach" podemos ver que uno de los registros que tiene es atípico. Este registro va a ser eliminado porque puede perjudicar el modelo. 

In [ ]:
result = subset[subset['thalach'] == 71]
result.head()

In [ ]:
subset = subset.drop(index=245)

In [78]:
fig = px.box(subset, y='thalach', )
fig.show()

In [ ]:
result2 = result = subset[subset['risk_score'] > 25]
result2.head(30)

Con el gráfico en barras podemos deducir que en la mayoria de datos del subconjunto es 0, es decir, que no se detectaron enfermedades. Y luego la cantidad de casos donde se empieza a detectar enfermedades se encuentran escalonadas, yendo desde el 1 (enfermedad leve) hasta 4 (enfermedad grave).

## 2- Análisis detallado de un conjunto de variables

### Variable de salida
**num:** Esta variable se encuentra dentro del csv ... 

In [ ]:
subset.num.value_counts().plot.bar(title='num')

### Variables de entrada

In [ ]:
subset.sex.value_counts().plot.bar(title='Sex (1: Hombre 0: Mujer)')

In [ ]:
mean = subset.thalach.mean()
stand = subset.thalach.std()

normal = np.random.normal(mean, stand, 1000)

plt.figure(figsize=(10,6))

sns.histplot(normal, kde=True, color='blue', bins=20)

plt.show()


In [ ]:
subset.age_group.value_counts().plot.bar(title='Grupo etario', )


### Variables Nulos
Estos son los valores nulos encontrados en el subset de datos:

In [ ]:
subset.isnull().sum()

#### Tratamiento de valores nulos

**Variables:**
- **ca:**
- **thal:**
- **age_group:**
- **cholesterol_level:**
- **bp_level:**
- **risk_factor:**
- **avg_chol_by_age_group:**

### Listado de posibles dudas/preguntas al encargado de proveer los datos

## 3- Hipótesis sobre los datos
### a. Formulación de hipótesis sobre la variable target bajo determinadas condiciones

### b. Comprobación de la hipótesis


### c. Creación de nuevas variables